<a href="https://colab.research.google.com/github/AbshkPskr/Zomato-Analysis/blob/notebook/z_graphs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [68]:
import plotly.graph_objs as go
import pandas as pd
import numpy as np


In [69]:
df = pd.read_csv("https://raw.githubusercontent.com/AbshkPskr/Zomato-Analysis/master/reviews.csv",names = ['name','rating','review','sentiment']).drop_duplicates()

df = df[df.name != 'n the latest in...read more"']

In [70]:
sc = open("req.txt","w")
sc.write('''h5py==2.9.0
Keras==1.1.0
numpy==1.16.0
pandas==0.24.1
python-dateutil==2.8.0
pytz==2018.9
PyYAML==5.1
scipy==1.2.1
six==1.12.0
Theano==1.0.4''')
sc.close()

In [71]:
!pip install -r req.txt -q

In [7]:
!pip freeze > req.txt
# !pip uninstall -r req.txt -y

In [72]:

import html
import pickle
import re

import pandas as pd
from keras import backend as K
from keras.models import load_model
from keras.preprocessing import sequence

import os;
os.environ['KERAS_BACKEND'] = 'theano'


class EmotionPredictor:
    def __init__(self, classification, setting, use_unison_model=True):
        """
        Args:
            classification (str): Either 'ekman', 'plutchik', 'poms'
                or 'unison'.
            setting (str): Either 'mc' or 'ml'.
            use_unison_model (bool): Whether to use unison model;
                else use single model.
        """
        if classification not in ['ekman', 'plutchik', 'poms', 'unison']:
            raise ValueError('Unknown emotion classification: {}'.format(
                classification))
        if setting not in ['mc', 'ml']:
            raise ValueError('Unknown setting: {}'.format(setting))

        self.classification = classification
        self.setting = setting
        self.use_unison_model = use_unison_model
        self.model = self._get_model()
        self.embeddings_model = self._get_embeddings_model()
        self.char_to_ind = self._get_char_mapping()
        self.class_values = self._get_class_values()
        self.max_len = self._get_max_sequence_length()

    def _get_model(self):
        self._loaded_model_filename = 'models/{}{}-{}.h5'.format(
            'unison-' if self.use_unison_model else '',
            self.classification,
            self.setting,
        )
        return load_model(self._loaded_model_filename)

    def _get_embeddings_model(self):
        last_layer_output = K.function([self.model.layers[0].input,
                                        K.learning_phase()],
                                       [self.model.layers[-3].output])
        return lambda x: last_layer_output([x, 0])[0]

    @staticmethod
    def _get_char_mapping():
        with open('models/allowed-chars.pkl', 'rb') as f:
            return pickle.load(f)

    def _get_class_values(self):
        if self.classification == 'ekman':
            return ['Anger', 'Disgust', 'Fear', 'Joy', 'Sadness', 'Surprise']
        elif self.classification == 'plutchik':
            return ['Anger', 'Disgust', 'Fear', 'Joy', 'Sadness', 'Surprise',
                    'Trust', 'Anticipation']
        elif self.classification == 'poms':
            return ['Anger', 'Depression', 'Fatigue', 'Vigour', 'Tension',
                    'Confusion']

    def _get_max_sequence_length(self):
        if self.use_unison_model or self.classification == 'poms':
            return 143
        elif self.classification in ['ekman', 'plutchik']:
            return 141

    def predict_classes(self, tweets):
        indices = self._tweet_to_indices(tweets)
        predictions = self.model.predict(indices, verbose=False)

        df = pd.DataFrame({'Tweet': tweets})
        if self.setting == 'mc':
            df['Emotion'] = [self.class_values[i] for i in
                        predictions.argmax(axis=-1)]
        else:
            predictions[predictions >= 0.5] = 1
            predictions[predictions < 0.5] = 0
            for emotion, values in zip(self.class_values, predictions.T):
                df[emotion] = values
        return df

    def predict_probabilities(self, tweets):
        indices = self._tweet_to_indices(tweets)
        predictions = self.model.predict(indices, verbose=False)

        df = pd.DataFrame({'Tweet': tweets})
        for emotion, values in zip(self.class_values, predictions.T):
            df[emotion] = values
        return df

    def embed(self, tweets):
        indices = self._tweet_to_indices(tweets)
        embeddings = self.embeddings_model(indices)

        df = pd.DataFrame({'Tweet': tweets})
        for index, values in enumerate(embeddings.T, start=1):
            df['Dim{}'.format(index)] = values
        return df

    def embedd(self, tweets):
        """ Here only for backwards compatibility. """
        return self.embed(tweets)

    def _tweet_to_indices(self, tweets):
        indices = []
        for t in tweets:
            t = html.unescape(t)                            # unescape HTML
            t = re.sub(r"http\S+", "", t)                   # remove normal URLS
            t = re.sub(r"pic\.twitter\.com/\S+", "", t)     # remove pic.twitter.com URLS
            indices.append([self.char_to_ind[char] for char in t])
        return sequence.pad_sequences(indices, maxlen=self.max_len)

In [ ]:
import pandas as pd

# Pandas presentation options
pd.options.display.max_colwidth = 150   # show whole tweet's content
pd.options.display.width = 200          # don't break columns
# pd.options.display.max_columns = 7      # maximal number of columns

# Predictor for Ekman's emotions in multiclass setting.
model = EmotionPredictor(classification='ekman', setting='mc', use_unison_model=True)
tweets = ["Watching the sopranos again from start to finish!"]

predictions = model.predict_classes(tweets)
print(predictions, '\n')

probabilities = model.predict_probabilities(tweets)
print(probabilities, '\n')

embeddings = model.embed(tweets)
print(embeddings, '\n')

In [60]:
hotel = 6
df.name.to_list()[0:hotel][hotel-1]

"Mama's Buoi"

In [74]:
model = EmotionPredictor(classification='plutchik', setting='mc', use_unison_model=False)

In [75]:
emotion = model.predict_classes(df['review'].to_list())

In [76]:
emotion.groupby('Emotion').count()

,Tweet
Emotion,
Anger,2
Anticipation,1
Disgust,2
Fear,5
Joy,17852
Sadness,37
Surprise,183
Trust,285


#Dash

In [5]:
bar = df.groupby('name').count().sort_values(by = 'rating')

In [ ]:
!pip install jupyter-dash -q
!pip install dash-dangerously-set-inner-html -q

In [21]:
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import dash
from  dash_dangerously_set_inner_html import DangerouslySetInnerHTML as DSIH

# Load Data
# df = px.data.tips()
# Build App
external_css = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = JupyterDash(__name__,external_stylesheets=external_css)

child1 = {'width': '46%','float': 'left','padding': '20px','border':'2px solid grey'}
child2 = {'width': '46%','float': 'right','padding': '20px','border':'2px solid grey'}

app.layout = html.Div([html.H1("js",style={'textAlign': 'center'}),
                       dcc.Dropdown(id='restaurant-dropdown',
                                    options=[{'label': i, 'value': i} for i in bar.index.unique()],
                                    value='Elation'),
                       html.Div(children=[
                                          html.Div(children = [dcc.Graph(id='graph')],
                                                   style=child1),
                                          html.Div(children = [dcc.Graph(id='radar')],
                                                   style=child2)
                                          ],
                                style={'border': '3px solid #fff','padding': '20px'}),
                    #html.Div(DSIH(df[df.name == 'Lost Lemons'].to_html())),
                       ])
# Define callback to update graph
@app.callback(
    Output('graph', 'figure'),
    Output('radar', 'figure'),
    [Input("restaurant-dropdown", "value")]
)

def update_figure(drop):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df[df.name == drop].index,
                             y=df[df.name == drop].sentiment,
                             mode='lines',
                             name='confirmed',line=dict( width=1)))
    fig.update_layout(margin=dict(l=0,r=20,b=0,t=100,pad=0),
                      paper_bgcolor="white",height= 500,
                      legend=dict(x=.01,y=.98),
                      title_text = 'reviews',font_size=15,
                      xaxis_title="name",
                      yaxis_title="Number of reviews")
    da = pd.DataFrame({'r':df[df.name == drop].sentiment,'theta':df[df.name == drop].index})
    radar = go.Figure()
    radar.add_trace(go.Scatterpolar(r=da['r'],theta=da['theta'],mode='lines',fill='toself'))
    radar.update_layout(height=500)
    return [fig,radar]

# if __name__ == '__main__':
app.run_server(debug=True,port = 8060)#,mode = "inline")

Dash app running on:


<IPython.core.display.Javascript object>

#Bokeh in Dash

In [8]:
# bokeh basics
from bokeh.plotting import figure
from bokeh.io import show, output_notebook

# Create a blank figure with labels
p = figure(plot_width = 600, plot_height = 600, 
           title = 'Example Glyphs',
           x_axis_label = 'X', y_axis_label = 'Y')

# Example data
squares_x = [1, 3, 4, 5, 8]
squares_y = [8, 7, 3, 1, 10]
circles_x = [9, 12, 4, 3, 15]
circles_y = [8, 4, 11, 6, 10]

# Add squares glyph
p.square(squares_x, squares_y, size = 12, color = 'navy', alpha = 0.6)
# Add circle glyph
p.circle(circles_x, circles_y, size = 12, color = 'red')

# Set to output the plot in the notebook
output_notebook()
# Show the plot
show(p)

In [9]:
# da = pd.DataFrame({'r':[1, 2, 3],'theta':['a', 'b', 'c']})
# radar = go.Figure()
# radar.add_trace(go.Scatterpolar(r=da['r'],theta=da['theta'],mode='lines',fill='toself'))
# radar.update_layout(height=500)

#Terminal

In [10]:
from IPython.display import JSON
from google.colab import output
from subprocess import getoutput
import os

def shell(command):
  if command.startswith('cd'):
    path = command.strip().split(maxsplit=1)[1]
    os.chdir(path)
    return JSON([''])
  return JSON([getoutput(command)])
output.register_callback('shell', shell)

In [11]:
%%html
<div id=term_demo></div>
<script src="https://code.jquery.com/jquery-latest.js"></script>
<script src="https://cdn.jsdelivr.net/npm/jquery.terminal/js/jquery.terminal.min.js"></script>
<link href="https://cdn.jsdelivr.net/npm/jquery.terminal/css/jquery.terminal.min.css" rel="stylesheet"/>
<script>
  $('#term_demo').terminal(async function(command) {
      if (command !== '') {
          try {
              let res = await google.colab.kernel.invokeFunction('shell', [command])
              let out = res.data['application/json'][0]
              this.echo(new String(out))
          } catch(e) {
              this.error(new String(e));
          }
      } else {
          this.echo('');
      }
  }, {
      greetings: 'Welcome to Colab Shell',
      name: 'colab_demo',
      height: 250,
      prompt: 'colab > '
  });

#HTML in Dash

In [12]:
import dash
from jupyter_dash import JupyterDash
import dash_html_components as html


class CustomDash(JupyterDash):
    def interpolate_index(self, **kwargs):
        # Inspect the arguments by printing them
        print(kwargs)
        return '''
        <!DOCTYPE html>
        <html>
            <head>
                <title>My App</title>
            </head>
            <body>
                <h1>Addition of two numbers</h1>
                <p id="demo2"></p>
                {app_entry}
                {config}
                {scripts}
                <script>
                var x=5;
                var y=6;
                var z;
                z=x+y;
                document.getElementById("demo2").innerHTML="Sum of 5 and 6 is "+ z;
                </script>
                {renderer}
                <div id="custom-footer">My custom footer</div>
            </body>
        </html>
        '''.format(
            app_entry=kwargs['app_entry'],
            config=kwargs['config'],
            scripts=kwargs['scripts'],
            renderer=kwargs['renderer'])

app = CustomDash()

app.layout = html.Div('Simple Dash App')

app.run_server(debug=True,port = 8060)


Dash app running on:


<IPython.core.display.Javascript object>

In [13]:
import dash
from jupyter_dash import JupyterDash
import dash_html_components as html


class CustomDash(JupyterDash):
    def interpolate_index(self, **kwargs):
        # Inspect the arguments by printing them
        print(kwargs)
        return '''
                <html>
                <head>
                <title>Simple Radar Chart</title>
                <link rel="stylesheet" href="style.css"/>
                <script src="https://mbostock.github.com/d3/d3.js?2.5.0"></script>
                <script src="radar.js"></script>
                </head><body><h1>Simple Radar Chart</h1>
                {app_entry}
                {config}
                {scripts}
                {renderer}
                <div id="viz">
                </div>
                <script>loadViz();</script>
                </body>
                </html>
        '''.format(
            app_entry=kwargs['app_entry'],
            config=kwargs['config'],
            scripts=kwargs['scripts'],
            renderer=kwargs['renderer'])

app = CustomDash()

app.layout = html.Div('Simpleasdfasdfasdf Dash App')
app.interpolate_index(app_entry=None,scripts=radarjs,config=None,renderer=None)


app.run_server(debug=True,port = 8060)


SyntaxError: ignored

In [ ]:
stylecss = '''.axis {
    shape-rendering: crispEdges;
}

.axis line {
    stroke: #000000;
    stroke-width: 2px;
}

.axis .ticks line {
    stroke: #4F4F4F;
    stroke-width: 2px;
}

.axis .ticks line.minor{
    stroke: #CCCCCC;
    stroke-width:1px;'''

sc = open("style.css","w")
sc.write(radarjs)
sc.close()

In [ ]:
radarjs = '''var series, 
    hours,
    minVal,
    maxVal,
    w = 400,
    h = 400,
    vizPadding = {
        top: 10,
        right: 0,
        bottom: 15,
        left: 0
    },
    radius,
    radiusLength,
    ruleColor = "#CCC";

var loadViz = function(){
  loadData();
  buildBase();
  setScales();
  addAxes();
  draw();
};

var loadData = function(){
    var randomFromTo = function randomFromTo(from, to){
       return Math.floor(Math.random() * (to - from + 1) + from);
    };

    series = [
      [],
      []
    ];

    hours = [];

    for (i = 0; i < 24; i += 1) {
        series[0][i] = randomFromTo(0,20);
        series[1][i] = randomFromTo(5,15);
        hours[i] = i; //in case we want to do different formatting
    }

    mergedArr = series[0].concat(series[1]);

    minVal = d3.min(mergedArr);
    maxVal = d3.max(mergedArr);
    //give 25% of range as buffer to top
    maxVal = maxVal + ((maxVal - minVal) * 0.25);
    minVal = 0;

    //to complete the radial lines
    for (i = 0; i < series.length; i += 1) {
        series[i].push(series[i][0]);
    }
};

var buildBase = function(){
    var viz = d3.select("#viz")
        .append('svg:svg')
        .attr('width', w)
        .attr('height', h)
        .attr('class', 'vizSvg');

    viz.append("svg:rect")
        .attr('id', 'axis-separator')
        .attr('x', 0)
        .attr('y', 0)
        .attr('height', 0)
        .attr('width', 0)
        .attr('height', 0);
    
    vizBody = viz.append("svg:g")
        .attr('id', 'body');
};

setScales = function () {
  var heightCircleConstraint,
      widthCircleConstraint,
      circleConstraint,
      centerXPos,
      centerYPos;

  //need a circle so find constraining dimension
  heightCircleConstraint = h - vizPadding.top - vizPadding.bottom;
  widthCircleConstraint = w - vizPadding.left - vizPadding.right;
  circleConstraint = d3.min([
      heightCircleConstraint, widthCircleConstraint]);

  radius = d3.scale.linear().domain([minVal, maxVal])
      .range([0, (circleConstraint / 2)]);
  radiusLength = radius(maxVal);

  //attach everything to the group that is centered around middle
  centerXPos = widthCircleConstraint / 2 + vizPadding.left;
  centerYPos = heightCircleConstraint / 2 + vizPadding.top;

  vizBody.attr("transform",
      "translate(" + centerXPos + ", " + centerYPos + ")");
};

addAxes = function () {
  var radialTicks = radius.ticks(5),
      i,
      circleAxes,
      lineAxes;

  vizBody.selectAll('.circle-ticks').remove();
  vizBody.selectAll('.line-ticks').remove();

  circleAxes = vizBody.selectAll('.circle-ticks')
      .data(radialTicks)
      .enter().append('svg:g')
      .attr("class", "circle-ticks");

  circleAxes.append("svg:circle")
      .attr("r", function (d, i) {
          return radius(d);
      })
      .attr("class", "circle")
      .style("stroke", ruleColor)
      .style("fill", "none");

  circleAxes.append("svg:text")
      .attr("text-anchor", "middle")
      .attr("dy", function (d) {
          return -1 * radius(d);
      })
      .text(String);

  lineAxes = vizBody.selectAll('.line-ticks')
      .data(hours)
      .enter().append('svg:g')
      .attr("transform", function (d, i) {
          return "rotate(" + ((i / hours.length * 360) - 90) +
              ")translate(" + radius(maxVal) + ")";
      })
      .attr("class", "line-ticks");

  lineAxes.append('svg:line')
      .attr("x2", -1 * radius(maxVal))
      .style("stroke", ruleColor)
      .style("fill", "none");

  lineAxes.append('svg:text')
      .text(String)
      .attr("text-anchor", "middle")
      .attr("transform", function (d, i) {
          return (i / hours.length * 360) < 180 ? null : "rotate(180)";
      });
};

var draw = function () {
  var groups,
      lines,
      linesToUpdate;

  highlightedDotSize = 4;

  groups = vizBody.selectAll('.series')
      .data(series);
  groups.enter().append("svg:g")
      .attr('class', 'series')
      .style('fill', function (d, i) {
          if(i === 0){
            return "green";
          } else {
            return "blue";
          }
      })
      .style('stroke', function (d, i) {
          if(i === 0){
            return "green";
          } else {
            return "blue";
          }
      });
  groups.exit().remove();

  lines = groups.append('svg:path')
      .attr("class", "line")
      .attr("d", d3.svg.line.radial()
          .radius(function (d) {
              return 0;
          })
          .angle(function (d, i) {
              if (i === 24) {
                  i = 0;
              } //close the line
              return (i / 24) * 2 * Math.PI;
          }))
      .style("stroke-width", 3)
      .style("fill", "none");

  groups.selectAll(".curr-point")
      .data(function (d) {
          return [d[0]];
      })
      .enter().append("svg:circle")
      .attr("class", "curr-point")
      .attr("r", 0);

  groups.selectAll(".clicked-point")
      .data(function (d) {
          return [d[0]];
      })
      .enter().append("svg:circle")
      .attr('r', 0)
      .attr("class", "clicked-point");

  lines.attr("d", d3.svg.line.radial()
      .radius(function (d) {
          return radius(d);
      })
      .angle(function (d, i) {
          if (i === 24) {
              i = 0;
          } //close the line
          return (i / 24) * 2 * Math.PI;
      }));
};'''

rr = open("radar.js","w")
rr.write(radarjs)
rr.close()